# 📦 13_load_playlist_items_manual_static_to_bigquery

## 🎯 Objetivo

Cargar la tabla estructural playlist_items_manual_static a BigQuery.

No es histórica. No contiene snapshot_date.  
Se reescribe completamente en cada ejecución del pipeline.  
Actúa como tabla puente entre playlists y videos.

Destino: youtube-datasets-360.angelgarciadatablog.playlist_items_manual_static

In [ ]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

In [ ]:
load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "playlist_items_manual_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)


## 🧱 Cargar snapshot desde Parquet (temporal)  

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [ ]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

df_playlist_items_manual_static = pd.read_parquet(
    PROCESSED_PATH / "playlist_items_manual_static.parquet"
)

df_playlist_items_manual_static.head()


In [ ]:
df_playlist_items_manual_static.dtypes

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 


In [ ]:
from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("playlist_id", "STRING"),
    SchemaField("video_id", "STRING"),
    SchemaField("position", "INT64"),
    SchemaField("added_at", "TIMESTAMP"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada.")



## 📌 Cargar datos del parquet a big query

In [ ]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = sobreescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_playlist_items_manual_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla playlist_items_manual_static reemplazada correctamente.")

